In [2]:
import pandas as pd

# Loading our csv
df = pd.read_csv("../data/raw/ed_v0_decisions.csv")

# A Look at our raw dataset
print(df)

print("\n--- BASIC INFO ---")
print(df.info())

print("\n--- ACTION COUNTS ---")
print(df["action"].value_counts())

print("\n--- DISTANCE COUNTS ---")
print(df["distance"].value_counts())

print("\n--- SCREEN COUNTS ---")
print(df["screen"].value_counts())

print("\n--- ACTION BY DISTANCE ---")
print(pd.crosstab(df["action"], df["distance"]))

     timestamp    phase                action distance     screen  \
0        7.400  neutral                  poke      mid  midscreen   
1        9.400  neutral                  poke      mid  midscreen   
2       10.700  neutral  whiff_punish_attempt      mid  midscreen   
3       11.500  neutral              approach      far  midscreen   
4       16.600  neutral                  poke      far  midscreen   
..         ...      ...                   ...      ...        ...   
130    605.486  neutral                  poke    close  cornering   
131    608.021  neutral          counter_poke      mid  cornering   
132    612.263  neutral  whiff_punish_attempt      mid  cornering   
133    614.363  neutral                  poke      mid  cornering   
134    622.208  neutral          counter_poke      mid  cornering   

              last_action            notes  
0                    none       roundstart  
1                    poke           poking  
2                    poke  whiff pun

In [3]:
from sklearn.preprocessing import LabelEncoder #To convert our labels to number
from sklearn.tree import DecisionTreeClassifier #Our model for smaller dataste 
from sklearn.model_selection import train_test_split #train test split
from sklearn.metrics import accuracy_score #Checking for accuracy

In [4]:
X = df[["distance", "screen"]] #Our Features
Y = df[["action"]] #Our target
X , Y

(    distance     screen
 0        mid  midscreen
 1        mid  midscreen
 2        mid  midscreen
 3        far  midscreen
 4        far  midscreen
 ..       ...        ...
 130    close  cornering
 131      mid  cornering
 132      mid  cornering
 133      mid  cornering
 134      mid  cornering
 
 [135 rows x 2 columns],
                    action
 0                    poke
 1                    poke
 2    whiff_punish_attempt
 3                approach
 4                    poke
 ..                    ...
 130                  poke
 131          counter_poke
 132  whiff_punish_attempt
 133                  poke
 134          counter_poke
 
 [135 rows x 1 columns])

In [5]:
#Converting both features and target into labels(numbers)
X_encoded = X.copy()  
enc_distance = LabelEncoder()
enc_screen = LabelEncoder()
enc_action = LabelEncoder()
X_encoded['distance'] = enc_distance.fit_transform(X['distance'])
X_encoded['screen'] = enc_screen.fit_transform(X['screen'])
Y_encoded = enc_action.fit_transform(Y)
X_encoded, Y_encoded



c:\Users\Shant\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\preprocessing\_label.py:120: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


(     distance  screen
 0           2       2
 1           2       2
 2           2       2
 3           1       2
 4           1       2
 ..        ...     ...
 130         0       1
 131         2       1
 132         2       1
 133         2       1
 134         2       1
 
 [135 rows x 2 columns],
 array([3, 3, 6, 0, 3, 2, 4, 3, 0, 3, 0, 3, 5, 3, 2, 4, 0, 1, 3, 5, 0, 4,
        2, 5, 3, 1, 4, 2, 2, 0, 3, 0, 0, 1, 3, 1, 1, 3, 2, 1, 0, 3, 1, 3,
        2, 3, 1, 1, 5, 2, 1, 0, 2, 0, 1, 2, 1, 3, 5, 0, 6, 0, 3, 6, 0, 1,
        3, 1, 5, 2, 0, 3, 3, 3, 0, 0, 1, 0, 5, 0, 3, 1, 5, 1, 0, 0, 1, 6,
        3, 1, 5, 6, 3, 3, 3, 5, 3, 3, 1, 3, 5, 3, 3, 1, 3, 3, 6, 6, 6, 6,
        5, 1, 3, 3, 1, 5, 3, 3, 6, 6, 1, 3, 3, 3, 3, 5, 6, 1, 3, 1, 3, 1,
        6, 3, 1]))

In [6]:
model = DecisionTreeClassifier(max_depth=3, random_state=42) #This is a basic if else state tree so for smaller data it makes sense right now
model.fit(X_encoded, Y_encoded) #Training our model

,"criterion criterion: {""gini"", ""entropy"", ""log_loss""}, default=""gini""The function to measure the quality of a split. Supported criteria are""gini"" for the Gini impurity and ""log_loss"" and ""entropy"" both for theShannon information gain, see :ref:`tree_mathematical_formulation`.",'gini'
,"splitter splitter: {""best"", ""random""}, default=""best""The strategy used to choose the split at each node. Supportedstrategies are ""best"" to choose the best split and ""random"" to choosethe best random split.",'best'
,"max_depth max_depth: int, default=NoneThe maximum depth of the tree. If None, then nodes are expanded untilall leaves are pure or until all leaves contain less thanmin_samples_split samples.",3
,"min_samples_split min_samples_split: int or float, default=2The minimum number of samples required to split an internal node:- If int, then consider `min_samples_split` as the minimum number.- If float, then `min_samples_split` is a fraction and `ceil(min_samples_split * n_samples)` are the minimum number of samples for each split... versionchanged:: 0.18 Added float values for fractions.",2
,"min_samples_leaf min_samples_leaf: int or float, default=1The minimum number of samples required to be at a leaf node.A split point at any depth will only be considered if it leaves atleast ``min_samples_leaf`` training samples in each of the left andright branches. This may have the effect of smoothing the model,especially in regression.- If int, then consider `min_samples_leaf` as the minimum number.- If float, then `min_samples_leaf` is a fraction and `ceil(min_samples_leaf * n_samples)` are the minimum number of samples for each node... versionchanged:: 0.18 Added float values for fractions.",1
,"min_weight_fraction_leaf min_weight_fraction_leaf: float, default=0.0The minimum weighted fraction of the sum total of weights (of allthe input samples) required to be at a leaf node. Samples haveequal weight when sample_weight is not provided.",0.0
,"max_features max_features: int, float or {""sqrt"", ""log2""}, default=NoneThe number of features to consider when looking for the best split:- If int, then consider `max_features` features at each split.- If float, then `max_features` is a fraction and `max(1, int(max_features * n_features_in_))` features are considered at each split.- If ""sqrt"", then `max_features=sqrt(n_features)`.- If ""log2"", then `max_features=log2(n_features)`.- If None, then `max_features=n_features`... note:: The search for a split does not stop until at least one valid partition of the node samples is found, even if it requires to effectively inspect more than ``max_features`` features.",None
,"random_state random_state: int, RandomState instance or None, default=NoneControls the randomness of the estimator. The features are alwaysrandomly permuted at each split, even if ``splitter`` is set to``""best""``. When ``max_features < n_features``, the algorithm willselect ``max_features`` at random at each split before finding the bestsplit among them. But the best found split may vary across differentruns, even if ``max_features=n_features``. That is the case, if theimprovement of the criterion is identical for several splits and onesplit has to be selected at random. To obtain a deterministic behaviourduring fitting, ``random_state`` has to be fixed to an integer.See :term:`Glossary ` for details.",42
,"max_leaf_nodes max_leaf_nodes: int, default=NoneGrow a tree with ``max_leaf_nodes`` in best-first fashion.Best nodes are defined as relative reduction in impurity.If None then unlimited number of leaf nodes.",None
,"min_impurity_decrease min_impurity_decrease: float, default=0.0A node will be split if this split induces a decrease of the impuritygreater than or equal to this value.The weighted impurity decrease equation is the following:: N_t / N * (impurity - N_t_R / N_t * right_impurity - N_t_L / N_t * left_impurity)where ``N`` is the total number of samples, ``N_t`` is the number ofsamples at the current no

In [7]:
print("v0 executed")
print(enc_action.classes_) #Everything our target could print out

v0 executed
['approach' 'counter_poke' 'jump' 'poke' 'retreat' 'wait'
 'whiff_punish_attempt']


In [8]:
#Test to prove that our model was atleast trained
def predict_action(distance, screen):
    d = enc_distance.transform([distance])[0]
    s = enc_screen.transform([screen])[0]
    
    pred = model.predict([[d, s]])[0]
    return enc_action.inverse_transform([pred])[0]

print("mid + midscreen →", predict_action("mid", "midscreen"))
print("close + cornering →", predict_action("close", "cornering"))
print("far + midscreen →", predict_action("far", "midscreen"))

mid + midscreen → poke
close + cornering → counter_poke
far + midscreen → poke


c:\Users\Shant\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\utils\validation.py:2691: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  warnings.warn(
c:\Users\Shant\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\utils\validation.py:2691: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  warnings.warn(
c:\Users\Shant\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\utils\validation.py:2691: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  warnings.warn(


In [9]:
X_train, X_test, Y_train, Y_test = train_test_split(
    X_encoded,
    Y_encoded,
    test_size=0.3,
    random_state=42
)
print(f"shape of inputs of training set : {X_train.shape}")
print(f"shape of outputs of training set : {Y_train.shape}")
print(f"shape of inputs of testing set : {X_test.shape}")
print(f"shape of outputs of testing set : {X_test.shape}")

shape of inputs of training set : (94, 2)
shape of outputs of training set : (94,)
shape of inputs of testing set : (41, 2)
shape of outputs of testing set : (41, 2)


In [10]:
#Model 0
model = DecisionTreeClassifier(max_depth=3, random_state=42)
model.fit(X_train, Y_train)

,"criterion criterion: {""gini"", ""entropy"", ""log_loss""}, default=""gini""The function to measure the quality of a split. Supported criteria are""gini"" for the Gini impurity and ""log_loss"" and ""entropy"" both for theShannon information gain, see :ref:`tree_mathematical_formulation`.",'gini'
,"splitter splitter: {""best"", ""random""}, default=""best""The strategy used to choose the split at each node. Supportedstrategies are ""best"" to choose the best split and ""random"" to choosethe best random split.",'best'
,"max_depth max_depth: int, default=NoneThe maximum depth of the tree. If None, then nodes are expanded untilall leaves are pure or until all leaves contain less thanmin_samples_split samples.",3
,"min_samples_split min_samples_split: int or float, default=2The minimum number of samples required to split an internal node:- If int, then consider `min_samples_split` as the minimum number.- If float, then `min_samples_split` is a fraction and `ceil(min_samples_split * n_samples)` are the minimum number of samples for each split... versionchanged:: 0.18 Added float values for fractions.",2
,"min_samples_leaf min_samples_leaf: int or float, default=1The minimum number of samples required to be at a leaf node.A split point at any depth will only be considered if it leaves atleast ``min_samples_leaf`` training samples in each of the left andright branches. This may have the effect of smoothing the model,especially in regression.- If int, then consider `min_samples_leaf` as the minimum number.- If float, then `min_samples_leaf` is a fraction and `ceil(min_samples_leaf * n_samples)` are the minimum number of samples for each node... versionchanged:: 0.18 Added float values for fractions.",1
,"min_weight_fraction_leaf min_weight_fraction_leaf: float, default=0.0The minimum weighted fraction of the sum total of weights (of allthe input samples) required to be at a leaf node. Samples haveequal weight when sample_weight is not provided.",0.0
,"max_features max_features: int, float or {""sqrt"", ""log2""}, default=NoneThe number of features to consider when looking for the best split:- If int, then consider `max_features` features at each split.- If float, then `max_features` is a fraction and `max(1, int(max_features * n_features_in_))` features are considered at each split.- If ""sqrt"", then `max_features=sqrt(n_features)`.- If ""log2"", then `max_features=log2(n_features)`.- If None, then `max_features=n_features`... note:: The search for a split does not stop until at least one valid partition of the node samples is found, even if it requires to effectively inspect more than ``max_features`` features.",None
,"random_state random_state: int, RandomState instance or None, default=NoneControls the randomness of the estimator. The features are alwaysrandomly permuted at each split, even if ``splitter`` is set to``""best""``. When ``max_features < n_features``, the algorithm willselect ``max_features`` at random at each split before finding the bestsplit among them. But the best found split may vary across differentruns, even if ``max_features=n_features``. That is the case, if theimprovement of the criterion is identical for several splits and onesplit has to be selected at random. To obtain a deterministic behaviourduring fitting, ``random_state`` has to be fixed to an integer.See :term:`Glossary ` for details.",42
,"max_leaf_nodes max_leaf_nodes: int, default=NoneGrow a tree with ``max_leaf_nodes`` in best-first fashion.Best nodes are defined as relative reduction in impurity.If None then unlimited number of leaf nodes.",None
,"min_impurity_decrease min_impurity_decrease: float, default=0.0A node will be split if this split induces a decrease of the impuritygreater than or equal to this value.The weighted impurity decrease equation is the following:: N_t / N * (impurity - N_t_R / N_t * right_impurity - N_t_L / N_t * left_impurity)where ``N`` is the total number of samples, ``N_t`` is the number ofsamples at the current no

In [11]:
y_pred = model.predict(X_test)
accuracy = accuracy_score(Y_test, y_pred)
print(f"Accuracy : {accuracy}")  #Will give out bad accuracy as the data is really small
print("\nPredictions vs actual:")
for actual, prediction in zip(Y_test, y_pred):
    print(enc_action.inverse_transform([actual])[0], "->", enc_action.inverse_transform([prediction])[0])

Accuracy : 0.2682926829268293

Predictions vs actual:
counter_poke -> poke
counter_poke -> poke
poke -> poke
wait -> poke
counter_poke -> poke
poke -> jump
wait -> poke
wait -> poke
wait -> poke
poke -> poke
jump -> counter_poke
approach -> counter_poke
jump -> poke
wait -> poke
whiff_punish_attempt -> poke
approach -> poke
approach -> poke
poke -> jump
approach -> poke
counter_poke -> counter_poke
approach -> poke
retreat -> poke
approach -> poke
poke -> poke
approach -> poke
counter_poke -> poke
poke -> poke
poke -> poke
poke -> poke
poke -> poke
poke -> poke
counter_poke -> poke
counter_poke -> poke
poke -> poke
wait -> poke
counter_poke -> poke
poke -> counter_poke
poke -> poke
whiff_punish_attempt -> poke
jump -> counter_poke
retreat -> poke


In [12]:
X_context = df[["distance", "screen", "last_action"]] #Adding more features
enc_last_action = LabelEncoder()

X_context_encoded = X_context.copy()
X_context_encoded["distance"] = enc_distance.fit_transform(X_context["distance"])
X_context_encoded["screen"] = enc_screen.fit_transform(X_context["screen"])
X_context_encoded["last_action"] = enc_last_action.fit_transform(X_context["last_action"])


In [13]:
X_train2, X_test2, Y_train2, Y_test2 = train_test_split(
    X_context_encoded, 
    Y_encoded, 
    test_size=0.3, 
    random_state=42
) #Splitting into train/test

model_context = DecisionTreeClassifier(max_depth=4, random_state=42)
model_context.fit(X_train2, Y_train2) #Training our model


,"criterion criterion: {""gini"", ""entropy"", ""log_loss""}, default=""gini""The function to measure the quality of a split. Supported criteria are""gini"" for the Gini impurity and ""log_loss"" and ""entropy"" both for theShannon information gain, see :ref:`tree_mathematical_formulation`.",'gini'
,"splitter splitter: {""best"", ""random""}, default=""best""The strategy used to choose the split at each node. Supportedstrategies are ""best"" to choose the best split and ""random"" to choosethe best random split.",'best'
,"max_depth max_depth: int, default=NoneThe maximum depth of the tree. If None, then nodes are expanded untilall leaves are pure or until all leaves contain less thanmin_samples_split samples.",4
,"min_samples_split min_samples_split: int or float, default=2The minimum number of samples required to split an internal node:- If int, then consider `min_samples_split` as the minimum number.- If float, then `min_samples_split` is a fraction and `ceil(min_samples_split * n_samples)` are the minimum number of samples for each split... versionchanged:: 0.18 Added float values for fractions.",2
,"min_samples_leaf min_samples_leaf: int or float, default=1The minimum number of samples required to be at a leaf node.A split point at any depth will only be considered if it leaves atleast ``min_samples_leaf`` training samples in each of the left andright branches. This may have the effect of smoothing the model,especially in regression.- If int, then consider `min_samples_leaf` as the minimum number.- If float, then `min_samples_leaf` is a fraction and `ceil(min_samples_leaf * n_samples)` are the minimum number of samples for each node... versionchanged:: 0.18 Added float values for fractions.",1
,"min_weight_fraction_leaf min_weight_fraction_leaf: float, default=0.0The minimum weighted fraction of the sum total of weights (of allthe input samples) required to be at a leaf node. Samples haveequal weight when sample_weight is not provided.",0.0
,"max_features max_features: int, float or {""sqrt"", ""log2""}, default=NoneThe number of features to consider when looking for the best split:- If int, then consider `max_features` features at each split.- If float, then `max_features` is a fraction and `max(1, int(max_features * n_features_in_))` features are considered at each split.- If ""sqrt"", then `max_features=sqrt(n_features)`.- If ""log2"", then `max_features=log2(n_features)`.- If None, then `max_features=n_features`... note:: The search for a split does not stop until at least one valid partition of the node samples is found, even if it requires to effectively inspect more than ``max_features`` features.",None
,"random_state random_state: int, RandomState instance or None, default=NoneControls the randomness of the estimator. The features are alwaysrandomly permuted at each split, even if ``splitter`` is set to``""best""``. When ``max_features < n_features``, the algorithm willselect ``max_features`` at random at each split before finding the bestsplit among them. But the best found split may vary across differentruns, even if ``max_features=n_features``. That is the case, if theimprovement of the criterion is identical for several splits and onesplit has to be selected at random. To obtain a deterministic behaviourduring fitting, ``random_state`` has to be fixed to an integer.See :term:`Glossary ` for details.",42
,"max_leaf_nodes max_leaf_nodes: int, default=NoneGrow a tree with ``max_leaf_nodes`` in best-first fashion.Best nodes are defined as relative reduction in impurity.If None then unlimited number of leaf nodes.",None
,"min_impurity_decrease min_impurity_decrease: float, default=0.0A node will be split if this split induces a decrease of the impuritygreater than or equal to this value.The weighted impurity decrease equation is the following:: N_t / N * (impurity - N_t_R / N_t * right_impurity - N_t_L / N_t * left_impurity)where ``N`` is the total number of samples, ``N_t`` is the number ofsamples at the current no

In [14]:
#Model 1
Y_pred2 = model_context.predict(X_test2)
context_acc = accuracy_score(Y_test2, Y_pred2)

print("Previous accuracy with just distance and screen :", accuracy)
print("Model accuracy with previous + last action:", context_acc)

df["last_action"].value_counts()


Previous accuracy with just distance and screen : 0.2682926829268293
Model accuracy with previous + last action: 0.4146341463414634


last_action
poke                    40
counter_poke            21
approach                20
wait                    15
none                    12
whiff_punish_attempt    11
jump                    11
retreat                  4
Name: count, dtype: int64

In [15]:
df["prev_distance"] = df["distance"].shift(1) #Making a new temporal feature to record the previous distance
df["prev_distance"] = df["prev_distance"].fillna("start") #To fill the first value as start
print(df[["distance","prev_distance"]].head(10))
df["prev_distance"].value_counts()

  distance prev_distance
0      mid         start
1      mid           mid
2      mid           mid
3      far           mid
4      far           far
5      far           far
6    close           far
7    close         close
8      mid         close
9    close           mid


prev_distance
mid      76
close    33
far      25
start     1
Name: count, dtype: int64

In [16]:
#Model 3
X_full = df[["distance", "screen", "last_action", "prev_distance"]]
enc_prev_distance = LabelEncoder()

X_full_encoded = X_full.copy()
X_full_encoded["distance"] = enc_distance.fit_transform(X_full["distance"])
X_full_encoded["screen"] = enc_screen.fit_transform(X_full["screen"])
X_full_encoded["last_action"] = enc_last_action.fit_transform(X_full["last_action"])
X_full_encoded["prev_distance"] = enc_prev_distance.fit_transform(X_full["prev_distance"])

X_train3, X_test3, Y_train3, Y_test3 = train_test_split(
    X_full_encoded,Y_encoded,test_size=0.3,random_state=42
)

model_full = DecisionTreeClassifier(max_depth=4, random_state=42)
model_full.fit(X_train3, Y_train3)

Y_pred3 = model_full.predict(X_test3)
full_acc = accuracy_score(Y_test3, Y_pred3)

print("Accuracy with new temportal feature prev_distance: ", full_acc)

Accuracy with new temportal feature prev_distance:  0.3170731707317073


In [ ]:
from sklearn.metrics import confusion_matrix, classification_report #Checking which of our features impact our 2nd model properly

cm = confusion_matrix(Y_test2, Y_pred2)
labels = enc_action.classes_
print("Confusion matrix:")
print(cm,"\n", labels)
print("\nClassification Report:")
print(classification_report(Y_test2, Y_pred2, target_names=labels))

Confusion matrix:
[[3 2 0 2 0 0 0]
 [0 5 0 3 0 0 0]
 [0 2 0 1 0 0 0]
 [3 1 2 6 0 1 0]
 [0 1 0 1 0 0 0]
 [0 1 0 1 0 3 1]
 [0 0 0 2 0 0 0]] 
 ['approach' 'counter_poke' 'jump' 'poke' 'retreat' 'wait'
 'whiff_punish_attempt']

Classification Report:
                      precision    recall  f1-score   support

            approach       0.50      0.43      0.46         7
        counter_poke       0.42      0.62      0.50         8
                jump       0.00      0.00      0.00         3
                poke       0.38      0.46      0.41        13
             retreat       0.00      0.00      0.00         2
                wait       0.75      0.50      0.60         6
whiff_punish_attempt       0.00      0.00      0.00         2

            accuracy                           0.41        41
           macro avg       0.29      0.29      0.28        41
        weighted avg       0.40      0.41      0.40        41



c:\Users\Shant\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\metrics\_classification.py:1833: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Users\Shant\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\metrics\_classification.py:1833: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Users\Shant\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\metrics\_classification.py:1833: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(ave